In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import gc
import math
import random
import pickle
import pandas as pd
import numpy as np
import multiprocessing
from tqdm import tqdm

from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, sampler
import torch.optim.lr_scheduler as lr_scheduler

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from imblearn.over_sampling import SMOTE, SMOTENC, SMOTEN, RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, CondensedNearestNeighbour, OneSidedSelection, NeighbourhoodCleaningRule

import shap
import catboost
from catboost import Pool, cv

pd.set_option('display.max_columns', 500)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
train = pd.read_csv("./data/train.csv").drop(columns=['father', 'mother', 'gender'])
train.drop_duplicates(subset=train.columns.tolist()[5:20], inplace=True, ignore_index=True)
test = pd.read_csv("./data/test.csv").drop(columns=['father', 'mother', 'gender'])

train.iloc[:, 1:-1] = train.iloc[:, 1:-1].astype('category')
test.iloc[:, 1:] = test.iloc[:, 1:].astype('category')

answer = np.zeros(len(test)) - 1

train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      248 non-null    object  
 1   trait   248 non-null    category
 2   SNP_01  248 non-null    category
 3   SNP_02  248 non-null    category
 4   SNP_03  248 non-null    category
 5   SNP_04  248 non-null    category
 6   SNP_05  248 non-null    category
 7   SNP_06  248 non-null    category
 8   SNP_07  248 non-null    category
 9   SNP_08  248 non-null    category
 10  SNP_09  248 non-null    category
 11  SNP_10  248 non-null    category
 12  SNP_11  248 non-null    category
 13  SNP_12  248 non-null    category
 14  SNP_13  248 non-null    category
 15  SNP_14  248 non-null    category
 16  SNP_15  248 non-null    category
 17  class   248 non-null    object  
dtypes: category(16), object(2)
memory usage: 9.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data colu

(None, None)

In [3]:
# text 형태의 categorical 변수들을 숫자형태로 변경

for i in tqdm(range(1, 15+1)) :
    target = str(i) if i >= 10 else "0"+str(i)
    try :   
        cols = sorted(train[f"SNP_{target}"].unique().tolist())  
        train[f"SNP_{target}"] = train[f"SNP_{target}"].map(lambda x : 0 if x==cols[0] else (1 if x==cols[1] else 2))
        test[f"SNP_{target}"] = test[f"SNP_{target}"].map(lambda x : 0 if x==cols[0] else (1 if x==cols[1] else 2))
    except :
        continue

train.info(), test.info()

100%|██████████| 15/15 [00:00<00:00, 1871.23it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      248 non-null    object  
 1   trait   248 non-null    category
 2   SNP_01  248 non-null    category
 3   SNP_02  248 non-null    category
 4   SNP_03  248 non-null    category
 5   SNP_04  248 non-null    category
 6   SNP_05  248 non-null    category
 7   SNP_06  248 non-null    category
 8   SNP_07  248 non-null    category
 9   SNP_08  248 non-null    category
 10  SNP_09  248 non-null    category
 11  SNP_10  248 non-null    category
 12  SNP_11  248 non-null    category
 13  SNP_12  248 non-null    category
 14  SNP_13  248 non-null    category
 15  SNP_14  248 non-null    category
 16  SNP_15  248 non-null    category
 17  class   248 non-null    object  
dtypes: category(16), object(2)
memory usage: 9.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data colu

(None, None)

In [4]:
X, y = train.iloc[:, 1:-1].to_numpy(), train['class'].map(lambda x : 0 if x=='A' else(1 if x=='B' else 2)).values
X_test = test.iloc[:,1:].to_numpy()

# Load the data
train_data = torch.from_numpy(X)
test_data = torch.from_numpy(X_test)

# Preprocess the data by one-hot encoding the categories
one_hot_data_01 = F.one_hot(train_data[:,:1]-1, num_classes=2).view(len(X), 2).float()
one_hot_data_02 = F.one_hot(train_data[:,1:], num_classes=3).view(len(X), 3*train_data[:,1:].size(1)).float()

one_hot_train = torch.concat([one_hot_data_01, one_hot_data_02], axis=1)

one_hot_data_01 = F.one_hot(test_data[:,:1]-1, num_classes=2).view(len(X_test), 2).float()
one_hot_data_02 = F.one_hot(test_data[:,1:], num_classes=3).view(len(X_test), 3*test_data[:,1:].size(1)).float()

one_hot_test = torch.concat([one_hot_data_01, one_hot_data_02], axis=1)

one_hot_train.shape, one_hot_test.shape

(torch.Size([248, 47]), torch.Size([175, 47]))

In [5]:
random_seed=333
strategy1 = {0 : 40, 1 : 70, 2 : 60}

under1 = RandomUnderSampler(sampling_strategy=strategy1, random_state=random_seed)
under2 = EditedNearestNeighbours()
under3 = RepeatedEditedNearestNeighbours()
under4 = AllKNN()
under5 = CondensedNearestNeighbour(random_state=random_seed)
under6 = OneSidedSelection(random_state=random_seed)
under7 = NeighbourhoodCleaningRule()

X, y = one_hot_train, train['class'].map(lambda x : 0 if x=='A' else (1 if x=='B' else 2)).values

X1, y1 = under1.fit_resample(X, y)
X2, y2 = under2.fit_resample(X, y)
X3, y3 = under3.fit_resample(X, y)
X4, y4 = under4.fit_resample(X, y)
X5, y5 = under5.fit_resample(X, y)
X6, y6 = under6.fit_resample(X, y)
X7, y7 = under7.fit_resample(X, y)

strategy2 = {0 : 100, 1 : 120, 2 : 110}

over1 = SMOTEN(sampling_strategy=strategy2, random_state=random_seed)
over2 = SMOTE(sampling_strategy=strategy2, random_state=random_seed)
over3 = RandomOverSampler(sampling_strategy=strategy2, random_state=random_seed)

X8, y8 = over1.fit_resample(X, y)
X9, y9 = over2.fit_resample(X, y)
X10, y10 = over3.fit_resample(X, y)

data = np.concatenate([one_hot_train, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10])
label = np.concatenate([y, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10])

data.shape, label.shape

((2360, 47), (2360,))

In [6]:
random_seed = 3833
strategy1 = {0 : 40, 1 : 70, 2 : 50}

under1 = RandomUnderSampler(sampling_strategy=strategy1, random_state=random_seed)
under2 = EditedNearestNeighbours()
under3 = RepeatedEditedNearestNeighbours()
under4 = AllKNN()
under5 = CondensedNearestNeighbour(random_state=random_seed)
under6 = OneSidedSelection(random_state=random_seed)
under7 = NeighbourhoodCleaningRule()

X, y = one_hot_train, train['class'].map(lambda x : 0 if x=='A' else (1 if x=='B' else 2)).values

X1, y1 = under1.fit_resample(X, y)
X2, y2 = under2.fit_resample(X, y)
X3, y3 = under3.fit_resample(X, y)
X4, y4 = under4.fit_resample(X, y)
X5, y5 = under5.fit_resample(X, y)
X6, y6 = under6.fit_resample(X, y)
X7, y7 = under7.fit_resample(X, y)

strategy2 = {0 : 1000, 1 : 1000, 2 : 1000}

over1 = SMOTEN(sampling_strategy=strategy2, random_state=random_seed)
over2 = SMOTE(sampling_strategy=strategy2, random_state=random_seed)
over3 = RandomOverSampler(sampling_strategy=strategy2, random_state=random_seed)

X8, y8 = over1.fit_resample(X, y)
X9, y9 = over2.fit_resample(X, y)
X10, y10 = over3.fit_resample(X, y)

data = np.concatenate([data, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10])
label = np.concatenate([label, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10])

data.shape, label.shape

((12482, 47), (12482,))

In [7]:
random_seed = 3587
strategy1 = {0 : 40, 1 : 70, 2 : 50}

under1 = RandomUnderSampler(sampling_strategy=strategy1, random_state=random_seed)
under2 = EditedNearestNeighbours()
under3 = RepeatedEditedNearestNeighbours()
under4 = AllKNN()
under5 = CondensedNearestNeighbour(random_state=random_seed)
under6 = OneSidedSelection(random_state=random_seed)
under7 = NeighbourhoodCleaningRule()

X, y = one_hot_train, train['class'].map(lambda x : 0 if x=='A' else (1 if x=='B' else 2)).values

X1, y1 = under1.fit_resample(X, y)
X2, y2 = under2.fit_resample(X, y)
X3, y3 = under3.fit_resample(X, y)
X4, y4 = under4.fit_resample(X, y)
X5, y5 = under5.fit_resample(X, y)
X6, y6 = under6.fit_resample(X, y)
X7, y7 = under7.fit_resample(X, y)

strategy2 = {0 : 1000, 1 : 1200, 2 : 1000}

over1 = SMOTEN(sampling_strategy=strategy2, random_state=random_seed)
over2 = SMOTE(sampling_strategy=strategy2, random_state=random_seed)
over3 = RandomOverSampler(sampling_strategy=strategy2, random_state=random_seed)

X8, y8 = over1.fit_resample(X, y)
X9, y9 = over2.fit_resample(X, y)
X10, y10 = over3.fit_resample(X, y)

data = np.concatenate([data, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10])
label = np.concatenate([label, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10])

data.shape, label.shape

((23205, 47), (23205,))

In [8]:
random_seed = 313
strategy1 = {0 : 40, 1 : 70, 2 : 50}

under1 = RandomUnderSampler(sampling_strategy=strategy1, random_state=random_seed)
under2 = EditedNearestNeighbours()
under3 = RepeatedEditedNearestNeighbours()
under4 = AllKNN()
under5 = CondensedNearestNeighbour(random_state=random_seed)
under6 = OneSidedSelection(random_state=random_seed)
under7 = NeighbourhoodCleaningRule()

X, y = one_hot_train, train['class'].map(lambda x : 0 if x=='A' else (1 if x=='B' else 2)).values

X1, y1 = under1.fit_resample(X, y)
X2, y2 = under2.fit_resample(X, y)
X3, y3 = under3.fit_resample(X, y)
X4, y4 = under4.fit_resample(X, y)
X5, y5 = under5.fit_resample(X, y)
X6, y6 = under6.fit_resample(X, y)
X7, y7 = under7.fit_resample(X, y)

strategy2 = {0 : 100, 1 : 120, 2 : 110}

over1 = SMOTEN(sampling_strategy=strategy2, random_state=random_seed)
over2 = SMOTE(sampling_strategy=strategy2, random_state=random_seed)
over3 = RandomOverSampler(sampling_strategy=strategy2, random_state=random_seed)

X8, y8 = over1.fit_resample(X, y)
X9, y9 = over2.fit_resample(X, y)
X10, y10 = over3.fit_resample(X, y)

data = np.concatenate([data, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10])
label = np.concatenate([label, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10])

data.shape, label.shape

((25327, 47), (25327,))

In [9]:
random_seed = 10000
strategy1 = {0 : 40, 1 : 70, 2 : 50}

under1 = RandomUnderSampler(sampling_strategy=strategy1, random_state=random_seed)
under2 = EditedNearestNeighbours()
under3 = RepeatedEditedNearestNeighbours()
under4 = AllKNN()
under5 = CondensedNearestNeighbour(random_state=random_seed)
under6 = OneSidedSelection(random_state=random_seed)
under7 = NeighbourhoodCleaningRule()

X, y = one_hot_train, train['class'].map(lambda x : 0 if x=='A' else (1 if x=='B' else 2)).values

X1, y1 = under1.fit_resample(X, y)
X2, y2 = under2.fit_resample(X, y)
X3, y3 = under3.fit_resample(X, y)
X4, y4 = under4.fit_resample(X, y)
X5, y5 = under5.fit_resample(X, y)
X6, y6 = under6.fit_resample(X, y)
X7, y7 = under7.fit_resample(X, y)

strategy2 = {0 : 100, 1 : 120, 2 : 110}

over1 = SMOTEN(sampling_strategy=strategy2, random_state=random_seed)
over2 = SMOTE(sampling_strategy=strategy2, random_state=random_seed)
over3 = RandomOverSampler(sampling_strategy=strategy2, random_state=random_seed)

X8, y8 = over1.fit_resample(X, y)
X9, y9 = over2.fit_resample(X, y)
X10, y10 = over3.fit_resample(X, y)

data = np.concatenate([data, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10])
label = np.concatenate([label, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10])

data.shape, label.shape

((27431, 47), (27431,))

In [29]:
df = pd.DataFrame(data=data, columns=[f"var_{x}" for x in range(data.shape[1])])
df[df >= 0.5] = 1.0
df[df < 0.5] = 0.0
df['class'] = label
df['var_2'] = 0.0
df.drop_duplicates(inplace=True, ignore_index=True)
df

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,class
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2
471,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2
472,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2
473,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [30]:
df.describe()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,class
count,475.000000,475.000000,475.0,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000
mean,0.229474,0.770526,0.0,0.202105,0.568421,0.183158,0.368421,0.357895,0.404211,0.336842,0.143158,0.480000,0.338947,0.124211,0.298947,0.338947,0.307368,0.189474,0.446316,0.223158,0.658947,0.181053,0.155789,0.286316,0.374737,0.294737,0.717895,0.187368,0.067368,0.143158,0.229474,0.602105,0.290526,0.330526,0.280000,0.541053,0.214737,0.162105,0.166316,0.349474,0.381053,0.722105,0.178947,0.067368,0.425263,0.298947,0.172632,1.090526
std,0.420938,0.420938,0.0,0.401994,0.495819,0.387204,0.482885,0.479886,0.491256,0.473129,0.350603,0.500127,0.473851,0.330169,0.458279,0.473851,0.461890,0.392298,0.497634,0.416802,0.474563,0.385468,0.363038,0.452515,0.484565,0.456405,0.450499,0.390618,0.250923,0.350603,0.420938,0.489980,0.454484,0.470899,0.449472,0.498837,0.411073,0.368936,0.372756,0.477306,0.486157,0.448434,0.383712,0.250923,0.494904,0.458279,0.378327,0.736492
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,1.000000,0.0,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,2.000000
max,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [31]:
class Transformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_classes):
        super(Transformer, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.num_classes = num_classes

        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads, dropout=0.5)
        self.output_layer = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.attention(x, x, x)[0]
        x = self.output_layer(x)
        return x


In [32]:
class SparseDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label


In [33]:
def train(model, data, num_epochs, batch_size, learning_rate):
    model = model.to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    step_size = 5
    gamma=0.9

    # define the scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Split the data into training and validation sets
    train_data, val_data = data, data

    # Create DataLoaders for the training and validation sets
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

    # Loop over the number of epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Loop over the training data
        for x, y in train_loader:
            # Move the data to the correct device
            x, y = x.to(device), torch.Tensor(y).to(device)

            # Forward pass
            logits = model(x)
            loss = criterion(logits, y)

            # Backward pass and optimization step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # update the learning rate
        scheduler.step()

        # Set the model to evaluation mode
        model.eval()

        # Initialize the validation loss and accuracy
        val_loss = 0.0
        val_acc = 0.0

        # Loop over the validation data
        for x, y in val_loader:
            # Move the data to the correct device
            x, y = x.to(device), y.to(device)

            # Forward pass
            logits = model(x)
            loss = criterion(logits, y)

            # Update the validation loss and accuracy
            val_loss += loss.item()
            val_acc += (logits.argmax(1) == y).float().mean().item()

        # Calculate the average validation loss and accuracy
        val_loss /= len(val_loader)
        val_acc /= len(val_loader)

        # Print the epoch, loss, and accuracy
        print(f"Epoch {epoch+1}: loss = {loss:.4f}, val_loss = {val_loss:.4f}, val_accuracy = {val_acc:.4f}")

    return model


In [34]:
# Define the model
modelA = Transformer(input_dim=47, hidden_dim=256, num_heads=4, num_classes=2)
modelB = Transformer(input_dim=47, hidden_dim=256, num_heads=8, num_classes=2)
modelC = Transformer(input_dim=47, hidden_dim=256, num_heads=8, num_classes=2)
modelBC = Transformer(input_dim=47, hidden_dim=128, num_heads=4, num_classes=2)

# Create the dataset and data loader
dataset_A = SparseDataset(df.iloc[:,:-1].values, (df['class']==0).values.astype(int), transform=None)
dataset_B = SparseDataset(df.iloc[:,:-1].values, (df['class']==1).values.astype(int), transform=None)
dataset_C = SparseDataset(df.iloc[:,:-1].values, (df['class']==2).values.astype(int), transform=None)
dataset_BC = SparseDataset(df[df['class']!=0].iloc[:,:-1].values, (df[df['class']!=0]['class']==2).values.astype(int), transform=None)

# Train the model
model_A = train(modelA, dataset_A, num_epochs=5, batch_size=64, learning_rate=0.001)

RuntimeError: mat1 and mat2 must have the same dtype

In [28]:
# Train the model
model_B = train(modelB, dataset_B, num_epochs=100, batch_size=512, learning_rate=0.001)

RuntimeError: mat1 and mat2 must have the same dtype

In [17]:
# Train the model
model_C = train(modelC, dataset_C, num_epochs=100, batch_size=4, learning_rate=0.001)

Epoch 1: loss = 0.3523, val_loss = 0.1780, val_accuracy = 0.9433
Epoch 2: loss = 0.0307, val_loss = 0.1176, val_accuracy = 0.9643
Epoch 3: loss = 0.0657, val_loss = 0.1051, val_accuracy = 0.9748
Epoch 4: loss = 0.2326, val_loss = 0.1542, val_accuracy = 0.9349
Epoch 5: loss = 0.0683, val_loss = 0.0867, val_accuracy = 0.9811
Epoch 6: loss = 0.0358, val_loss = 0.1124, val_accuracy = 0.9769
Epoch 7: loss = 0.0890, val_loss = 0.0951, val_accuracy = 0.9769
Epoch 8: loss = 0.0779, val_loss = 0.0725, val_accuracy = 0.9811
Epoch 9: loss = 0.1351, val_loss = 0.0927, val_accuracy = 0.9811
Epoch 10: loss = 0.2642, val_loss = 0.1237, val_accuracy = 0.9706
Epoch 11: loss = 0.1464, val_loss = 0.1029, val_accuracy = 0.9706
Epoch 12: loss = 0.3545, val_loss = 0.1806, val_accuracy = 0.9811
Epoch 13: loss = 0.0965, val_loss = 0.0464, val_accuracy = 0.9853
Epoch 14: loss = 0.0518, val_loss = 0.0486, val_accuracy = 0.9790
Epoch 15: loss = 0.1621, val_loss = 0.0466, val_accuracy = 0.9832
Epoch 16: loss = 0.

In [18]:
# Train the model
model_BC = train(modelBC, dataset_BC, num_epochs=125, batch_size=4, learning_rate=0.001)

Epoch 1: loss = 0.5196, val_loss = 0.4557, val_accuracy = 0.8859
Epoch 2: loss = 0.2758, val_loss = 0.1008, val_accuracy = 0.9701
Epoch 3: loss = 0.0770, val_loss = 0.0842, val_accuracy = 0.9728
Epoch 4: loss = 0.0131, val_loss = 0.0895, val_accuracy = 0.9647
Epoch 5: loss = 0.0144, val_loss = 0.0528, val_accuracy = 0.9810
Epoch 6: loss = 0.0185, val_loss = 0.0590, val_accuracy = 0.9755
Epoch 7: loss = 0.0105, val_loss = 0.0525, val_accuracy = 0.9755
Epoch 8: loss = 0.0015, val_loss = 0.0478, val_accuracy = 0.9837
Epoch 9: loss = 0.0311, val_loss = 0.1143, val_accuracy = 0.9620
Epoch 10: loss = 0.0158, val_loss = 0.0586, val_accuracy = 0.9810
Epoch 11: loss = 0.0306, val_loss = 0.0559, val_accuracy = 0.9837
Epoch 12: loss = 0.0243, val_loss = 0.0824, val_accuracy = 0.9755
Epoch 13: loss = 0.0198, val_loss = 0.0466, val_accuracy = 0.9918
Epoch 14: loss = 0.0018, val_loss = 0.0624, val_accuracy = 0.9837
Epoch 15: loss = 0.0063, val_loss = 0.0484, val_accuracy = 0.9837
Epoch 16: loss = 0.

In [19]:
X_test = one_hot_test.to(device)

pred_A = torch.softmax(model_A(X_test), axis=1)[:,1].detach().cpu().numpy()
pred_B = torch.softmax(model_B(X_test), axis=1)[:,1].detach().cpu().numpy()
pred_C = torch.softmax(model_C(X_test), axis=1)[:,1].detach().cpu().numpy()
pred_BC = torch.softmax(model_BC(X_test), axis=1)[:,1].detach().cpu().numpy()

In [20]:
submit = pd.read_csv("submit_high1.csv")
submit['A_prob'] = np.round(pred_A, 4)
submit['B_prob'] = np.round(pred_B, 4)
submit['C_prob'] = np.round(pred_C, 4)
submit['BC_prob'] = np.round(pred_BC, 4)
# submit['class'] = submit['class'].map(lambda x : 'A' if x==0 else ('B' if x==1 else 'C'))
submit['total'] = np.argmax(submit[['A_prob', 'B_prob', 'C_prob']].values, axis=1)
submit['total'] = submit['total'].map(lambda x : 'A' if x==0 else ('B' if x==1 else 'C'))
submit

,id,class,A_prob,B_prob,C_prob,BC_prob,total
0,TEST_000,A,1.0000,0.0000,0.0000,1.0000,A
1,TEST_001,B,0.0000,1.0000,0.0000,0.0000,B
2,TEST_002,C,0.0001,0.0000,1.0000,1.0000,C
3,TEST_003,C,0.0000,0.7512,0.9843,0.0129,C
4,TEST_004,A,1.0000,0.0000,0.0000,1.0000,A
...,...,...,...,...,...,...,...
170,TEST_170,B,0.0000,1.0000,0.0000,0.0000,B
171,TEST_171,C,0.0000,0.0000,1.0000,1.0000,C
172,TEST_172,C,0.0000,0.0000,1.0000,1.0000,C
173,TEST_173,B,0.0000,1.0000,0.0000,0.0000,B


In [21]:
submit[submit['class'] != submit.total]

,id,class,A_prob,B_prob,C_prob,BC_prob,total
5,TEST_005,C,0.0,1.0000,0.0000,0.0000,B
12,TEST_012,B,0.0,0.0000,0.4513,0.0001,C
127,TEST_127,B,0.0,0.0000,0.0000,0.0000,A
162,TEST_162,C,0.0,0.9983,0.0000,1.0000,B


In [22]:
submit['total2'] = 1
submit.loc[submit.BC_prob >= 0.6, 'total2'] = 2
submit.loc[submit.A_prob >= 0.5, 'total2'] = 0
submit['total2'] = submit['total2'].map(lambda x : 'A' if x==0 else ('B' if x==1 else 'C'))
submit[submit['class'] != submit.total2]

,id,class,A_prob,B_prob,C_prob,BC_prob,total,total2
3,TEST_003,C,0.0000,0.7512,0.9843,0.0129,C,B
5,TEST_005,C,0.0000,1.0000,0.0000,0.0000,B,B
103,TEST_103,C,0.0000,0.0000,1.0000,0.0000,C,B
126,TEST_126,B,0.0001,0.9493,0.0000,1.0000,B,C


In [23]:
submit['total'].value_counts()

B    84
A    52
C    39
Name: total, dtype: int64

In [24]:
submit['last1'] = pd.read_csv("submit_last1.csv")['class']
submit['last2'] = pd.read_csv("submit_last2.csv")['class']

submit[submit.total != submit.last2]

,id,class,A_prob,B_prob,C_prob,BC_prob,total,total2,last1,last2
127,TEST_127,B,0.0,0.0,0.0,0.0,A,B,C,B


In [26]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       175 non-null    object 
 1   class    175 non-null    object 
 2   A_prob   175 non-null    float32
 3   B_prob   175 non-null    float32
 4   C_prob   175 non-null    float32
 5   BC_prob  175 non-null    float32
 6   total    175 non-null    object 
 7   total2   175 non-null    object 
 8   last1    175 non-null    object 
 9   last2    175 non-null    object 
dtypes: float32(4), object(6)
memory usage: 11.1+ KB


In [27]:
df = pd.read_csv("submit_high1.csv")
df['class'] = submit['total']
df.to_csv("submit_last011.csv", index=False)
df

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,C
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B


In [28]:
df['class'].value_counts()

B    84
A    52
C    39
Name: class, dtype: int64

In [29]:
df = pd.read_csv("submit_high1.csv")
df['class'] = submit['total2']
df.to_csv("submit_last022.csv", index=False)
df

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B


In [30]:
df['class'].value_counts()

B    86
A    51
C    38
Name: class, dtype: int64